# AI Algorithms
> By Sayed Afnan Khazi(01JST21CB036), Venkat Bhaskar(01JST21CB049), G Rutvik(01JST21CB012), Sai Sujith(01JST21CB033)
- This notebook showcases the implementation of Artificial Neural Networks, K-Nearest Neighbors, Decision Trees, Random Forest, Support Vector Machines, and Reinforcement Learning (Q-learning) on a financial dataset containing Alibaba's stock prices for the past many years.

### Importing Libraries

In [1]:
# Install all the requirements
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importing all the requirements. 
import numpy as np
import pandas as pd
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

2024-05-31 00:00:50.210600: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
final_results_table = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

### Algorithm 1: Artificial Neural Networks

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping

# Load data from a CSV file
df = pd.read_csv('dataset.csv')

# Preprocess data
# Handling missing values (if any)
df = df.dropna()

# Convert categorical columns to numerical values using one-hot encoding
categorical_columns = ['Gender', 'Own_car', 'Own_property', 'Work_phone', 'Phone', 'Email', 'Income_type', 'Education_type', 'Family_status', 'Housing_type', 'Occupation_type']
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Feature selection
features = df.columns.difference(['ID', 'Target'])
X = df[features]
y = df['Target']

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

ratios = [0.4, 0.3, 0.2, 0.1]
results = {}

for ratio in ratios:
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=ratio, random_state=42, stratify=y_res)

    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Initialize the ANN
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=32, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the ANN
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the ANN
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=0)

    # Predictions
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix:\n", cm)

    # Calculate metrics
    precision_q = precision_score(y_test, y_pred, zero_division=0)
    accuracy_q = accuracy_score(y_test, y_pred)
    recall_q = recall_score(y_test, y_pred, zero_division=0)
    f1_q = f1_score(y_test, y_pred, zero_division=0)

    results[ratio] = {'precision': precision_q, 'accuracy': accuracy_q, 'recall': recall_q, 'f1': f1_q}

# Display results
for ratio, metrics in results.items():
    print(f"\nMetrics for fold {ratio}:")
    print(f"Precision: {metrics['precision']:.2f}")
    print(f"Accuracy: {metrics['accuracy']:.2f}")
    print(f"Recall: {metrics['recall']:.2f}")
    print(f"F1 Score: {metrics['f1']:.2f}")


# Detailed classification report for the last test size ratio
print("\nClassification Report for the last ratio:\n", classification_report(y_test, y_pred, zero_division=0))

# Get the most recent data point and format it as a DataFrame
last_data_point = df[features].iloc[-1].to_frame().T

# Standardize the last data point
last_data_point_scaled = scaler.transform(last_data_point)

# Predict the eligibility for the next data point
predicted_movement = model.predict(last_data_point_scaled)
predicted_movement = (predicted_movement > 0.5).astype(int)

# Interpret the result
movement_label = "Eligible" if predicted_movement[0][0] == 1 else "Not Eligible"
print(f'The predicted eligibility for the next person is: {movement_label}')


# Print results
print("Results:")
print("Ratio Precision Accuracy Recall F1")
for ratio, metrics in results.items():
    print(f"{ratio:}\t{metrics['precision']:.4f}\t{metrics['accuracy']:.4f}\t{metrics['recall']:.4f}\t{metrics['f1']:.4f}")
print("Average F1",round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Accuracy",round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Precision",round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Recall",round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4))

final_results_table.loc[len(final_results_table)] = ['ANN', 
                                                     round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4)
                                                    ]

211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Confusion Matrix:
 [[3347   24]
 [ 694 2676]]
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Confusion Matrix:
 [[2504   24]
 [ 526 2002]]
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Confusion Matrix:
 [[1678    8]
 [ 360 1325]]
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Confusion Matrix:
 [[838   5]
 [186 657]]

Metrics for fold 0.4:
Precision: 0.99
Accuracy: 0.89
Recall: 0.79
F1 Score: 0.88

Metrics for fold 0.3:
Precision: 0.99
Accuracy: 0.89
Recall: 0.79
F1 Score: 0.88

Metrics for fold 0.2:
Precision: 0.99
Accuracy: 0.89
Recall: 0.79
F1 Score: 0.88

Metrics for fold 0.1:
Precision: 0.99
Accuracy: 0.89
Recall: 0.78
F1 Score: 0.87

Classification Report for the last ratio:
               precision    recall  f1-score   support

           0       0.82      0.99      0.90       843
           1       0.99      0.78      0.87       843

    accuracy                           0.89      1686
   macro avg       0.91      0.89      0.89      1686
weighted avg  

### Algorithm 2: K-Nearest Neighbors

In [ ]:
# Load data from a CSV file
df = pd.read_csv('dataset.csv')

# Preprocess data
# Handling missing values (if any)
df = df.dropna()

# Convert categorical columns to numerical values using one-hot encoding
categorical_columns = ['Gender', 'Own_car', 'Own_property', 'Work_phone', 'Phone', 'Email', 'Income_type', 'Education_type', 'Family_status', 'Housing_type', 'Occupation_type']
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Feature selection
# Assuming 'Target' is the target variable
features = df.columns.difference(['ID', 'Target'])
X = df[features]
y = df['Target']

# Training-testing ratios
ratios = [0.4, 0.3, 0.2, 0.1]

results = {}

for ratio in ratios:
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=0)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Initialize and train the KNN classifier
    knn = KNeighborsClassifier(n_neighbors=2)
    knn.fit(X_train, y_train)
    
    # Predictions
    y_pred = knn.predict(X_test)
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for ratio {ratio}:\n", cm)
    
    # Calculate metrics
    precision_q = precision_score(y_test, y_pred, zero_division=0)
    accuracy_q = accuracy_score(y_test, y_pred)
    recall_q = recall_score(y_test, y_pred, zero_division=0)
    f1_q = f1_score(y_test, y_pred, zero_division=0)
    
    results[ratio] = {'precision': precision_q, 'accuracy': accuracy_q, 'recall': recall_q, 'f1': f1_q}

# Display results
for ratio, metrics in results.items():
    print(f"\nMetrics for test size ratio {ratio}:")
    print(f"Precision: {metrics['precision']:.2f}")
    print(f"Accuracy: {metrics['accuracy']:.2f}")
    print(f"Recall: {metrics['recall']:.2f}")
    print(f"F1 Score: {metrics['f1']:.2f}")

# Detailed classification report for the last test size ratio
print("\nClassification Report for the last ratio:\n", classification_report(y_test, y_pred, zero_division=0))

# Get the most recent data point and format it as a DataFrame
last_data_point = df[features].iloc[-1].to_frame().T

# Standardize the last data point
last_data_point_scaled = scaler.transform(last_data_point)

# Predict the eligibility for the next data point
predicted_eligibility = knn.predict(last_data_point_scaled)

# Interpret the result
eligibility_label = "Eligible" if predicted_eligibility[0] == 1 else "Not Eligible"
print(f'The predicted eligibility for the next person is: {eligibility_label}')


# Print results
print("Results:")
print("Ratio Precision Accuracy Recall F1")
for ratio, metrics in results.items():
    print(f"{ratio:}\t{metrics['precision']:.4f}\t{metrics['accuracy']:.4f}\t{metrics['recall']:.4f}\t{metrics['f1']:.4f}")
print("Average F1",round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Accuracy",round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Precision",round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Recall",round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4))

final_results_table.loc[len(final_results_table)] = ['KNN', 
                                                     round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4)
                                                    ]

### Algorithm 3: Decision Trees

In [ ]:
# Load data from the new CSV file
df = pd.read_csv('dataset.csv')

# List of categorical features to encode
categorical_features = ['Income_type', 'Education_type', 'Family_status', 'Housing_type', 'Occupation_type']

# Encode categorical features
label_encoders = {}
for feature in categorical_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    label_encoders[feature] = le

# Feature selection - excluding 'ID' and 'Target'
features = [col for col in df.columns if col not in ['ID', 'Target']]
X = df[features]
y = df['Target']

# Training-testing ratios
ratios = [0.4, 0.3, 0.2, 0.1]

results = {}

for ratio in ratios:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=0)

    # Model training
    classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)

    # Predictions
    y_pred = classifier.predict(X_test)

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for ratio {ratio}:\n", cm)

    # Calculate metrics
    precision_q = precision_score(y_test, y_pred)
    accuracy_q = accuracy_score(y_test, y_pred)
    recall_q = recall_score(y_test, y_pred)
    f1_q = f1_score(y_test, y_pred)
    
    results[ratio] = {'precision': precision_q, 'accuracy': accuracy_q, 'recall': recall_q, 'f1': f1_q}

# Detailed classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Get the most recent data point and format it as a DataFrame
last_data_point = df[features].iloc[-1].to_frame().T

# Predict the eligibility for the next individual
predicted_eligibility = classifier.predict(last_data_point)

# Interpret the result
eligibility_label = "Eligible" if predicted_eligibility[0] == 1 else "Not Eligible"
print(f'The predicted eligibility for the next individual is: {eligibility_label}')

# Print results
print("Results:")
print("Ratio Precision Accuracy Recall F1")
for ratio, metrics in results.items():
    print(f"{ratio:}\t{metrics['precision']:.4f}\t{metrics['accuracy']:.4f}\t{metrics['recall']:.4f}\t{metrics['f1']:.4f}")
print("Average F1",round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Accuracy",round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Precision",round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Recall",round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4))

final_results_table.loc[len(final_results_table)] = ['Decision Tree', 
                                                     round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4)
                                                    ]

### Algorithm 4: Random Forest

In [ ]:
# Load data from the new CSV file
df = pd.read_csv('dataset.csv')

# List of categorical features to encode
categorical_features = ['Income_type', 'Education_type', 'Family_status', 'Housing_type', 'Occupation_type']

# Encode categorical features
label_encoders = {}
for feature in categorical_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    label_encoders[feature] = le

# Feature selection - excluding 'ID' and 'Target'
features = [col for col in df.columns if col not in ['ID', 'Target']]
X = df[features]
y = df['Target']

# Training-testing ratios
ratios = [0.4, 0.3, 0.2, 0.1]

results = {}

for ratio in ratios:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=0)

    # Model training
    classifier = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)

    # Predictions
    y_pred = classifier.predict(X_test)

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for ratio {ratio}:\n", cm)

    # Calculate metrics
    precision_q = precision_score(y_test, y_pred)
    accuracy_q = accuracy_score(y_test, y_pred)
    recall_q = recall_score(y_test, y_pred)
    f1_q = f1_score(y_test, y_pred)
    
    results[ratio] = {'precision': precision_q, 'accuracy': accuracy_q, 'recall': recall_q, 'f1': f1_q}

# Detailed classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Get the most recent data point and format it as a DataFrame
last_data_point = df[features].iloc[-1].to_frame().T

# Predict the eligibility for the next individual
predicted_eligibility = classifier.predict(last_data_point)

# Interpret the result
eligibility_label = "Eligible" if predicted_eligibility[0] == 1 else "Not Eligible"
print(f'The predicted eligibility for the next individual is: {eligibility_label}')

# Print results
print("Results:")
print("Ratio Precision Accuracy Recall F1")
for ratio, metrics in results.items():
    print(f"{ratio:}\t{metrics['precision']:.4f}\t{metrics['accuracy']:.4f}\t{metrics['recall']:.4f}\t{metrics['f1']:.4f}")
print("Average F1",round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Accuracy",round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Precision",round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Recall",round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4))

final_results_table.loc[len(final_results_table)] = ['Random Forest', 
                                                     round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4)
                                                    ]

### Algorithm 5: Support Vector Machines

In [ ]:
# Load data from a CSV file
df = pd.read_csv('dataset.csv')

# Preprocess data
# Handling missing values (if any)
df = df.dropna()

# Convert categorical columns to numerical values using one-hot encoding
categorical_columns = ['Gender', 'Own_car', 'Own_property', 'Work_phone', 'Phone', 'Email', 'Income_type', 'Education_type', 'Family_status', 'Housing_type', 'Occupation_type']
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Feature selection
# Assuming 'Target' is the target variable
features = df.columns.difference(['ID', 'Target'])
X = df[features]
y = df['Target']

# Training-testing ratios
ratios = [0.4, 0.3, 0.2, 0.1]

results = {}

for ratio in ratios:
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=0)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Initialize and train the SVM classifier
    svm_classifier = SVC(kernel='sigmoid', random_state=0)
    svm_classifier.fit(X_train, y_train)
    
    # Predictions
    y_pred = svm_classifier.predict(X_test)
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for ratio {ratio}:\n", cm)
    
    # Calculate metrics
    precision_q = precision_score(y_test, y_pred, zero_division=0)
    accuracy_q = accuracy_score(y_test, y_pred)
    recall_q = recall_score(y_test, y_pred, zero_division=0)
    f1_q = f1_score(y_test, y_pred, zero_division=0)
    
    results[ratio] = {'precision': precision_q, 'accuracy': accuracy_q, 'recall': recall_q, 'f1': f1_q}

# Display results
for ratio, metrics in results.items():
    print(f"\nMetrics for test size ratio {ratio}:")
    print(f"Precision: {metrics['precision']:.2f}")
    print(f"Accuracy: {metrics['accuracy']:.2f}")
    print(f"Recall: {metrics['recall']:.2f}")
    print(f"F1 Score: {metrics['f1']:.2f}")

# Detailed classification report for the last test size ratio
print("\nClassification Report for the last ratio:\n", classification_report(y_test, y_pred, zero_division=0))

# Get the most recent data point and format it as a DataFrame
last_data_point = df[features].iloc[-1].to_frame().T

# Standardize the last data point
last_data_point_scaled = scaler.transform(last_data_point)

# Predict the eligibility for the next data point
predicted_eligibility = svm_classifier.predict(last_data_point_scaled)

# Interpret the result
eligibility_label = "Eligible" if predicted_eligibility[0] == 1 else "Not Eligible"
print(f'The predicted eligibility for the next data point is: {eligibility_label}')

# Print results
print("Results:")
print("Ratio Precision Accuracy Recall F1")
for ratio, metrics in results.items():
    print(f"{ratio:}\t{metrics['precision']:.4f}\t{metrics['accuracy']:.4f}\t{metrics['recall']:.4f}\t{metrics['f1']:.4f}")
print("Average F1",round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Accuracy",round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Precision",round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Recall",round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4))

final_results_table.loc[len(final_results_table)] = ['SVM',
                                                     round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4)
                                                    ]

### Algorithm 6: Reinforcement Learning (Q-learning)

In [ ]:
# Seed for reproducibility
random.seed(42)
np.random.seed(42)

# Load data from a CSV file
file_path = 'dataset.csv'
df = pd.read_csv(file_path)


# Preprocess data
# Handling missing values (if any)
df = df.dropna()

# Convert categorical columns to numerical values using one-hot encoding
categorical_columns = ['Gender', 'Own_car', 'Own_property', 'Work_phone', 'Phone', 'Email', 'Income_type', 'Education_type', 'Family_status', 'Housing_type', 'Occupation_type']
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Feature selection
# Assuming 'Target' is the target variable
features = df.columns.difference(['ID', 'Target'])
X = df[features]
y = df['Target']

# Q-learning algorithm
def q_learning_train(X, y, episodes=1000, learning_rate=0.1, discount_factor=0.95, epsilon=0.1):
    n_actions = 2  # Eligible or Not Eligible (0 or 1)
    n_states = X.shape[0]
    
    # Initialize Q-table with zeros
    Q = np.zeros((n_states, n_actions))
    
    for _ in range(episodes):
        state = random.randint(0, n_states - 1)
        while True:
            if random.uniform(0, 1) < epsilon:  # epsilon is our exploration rate
                action = random.randint(0, n_actions - 1)  # Explore
            else:
                action = np.argmax(Q[state, :])  # Exploit
            
            reward = 1 if y.iloc[state] == action else -1
            
            next_state = (state + 1) % n_states
            Q[state, action] = Q[state, action] + learning_rate * (reward + discount_factor * np.max(Q[next_state, :]) - Q[state, action])
            
            state = next_state
            if state == 0:
                break
    
    return Q

def q_learning_predict(Q, X):
    '''Predicts the actions for each state in X (test data) using the Q-table.'''
    y_pred = []
    for state in range(X.shape[0]):
        action = np.argmax(Q[state, :])
        y_pred.append(action)
    return np.array(y_pred)

def print_confusion_matrix(cm, title):
    print(f"{title}")
    print(cm,end='\n\n')

# training-testing ratios
ratios = [0.4, 0.3, 0.2, 0.1]

results = {}

for ratio in ratios:
    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=42)

    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Train Q-learning model
    Q = q_learning_train(pd.DataFrame(X_train), y_train)
    y_pred_q = q_learning_predict(Q, pd.DataFrame(X_test))

    # Print confusion matrix
    cm = confusion_matrix(y_test, y_pred_q)
    print_confusion_matrix(cm, f"Confusion Matrix for ratio {ratio}")

    # Calculate metrics
    precision_q = precision_score(y_test, y_pred_q, zero_division=1)
    accuracy_q = accuracy_score(y_test, y_pred_q)
    recall_q = recall_score(y_test, y_pred_q)
    f1_q = f1_score(y_test, y_pred_q)
    
    results[ratio] = {'precision': precision_q, 'accuracy': accuracy_q, 'recall': recall_q, 'f1': f1_q}

# Print results
for ratio in results:
    print(f"Results for test size ratio {ratio}:")
    for metric, value in results[ratio].items():
        print(f"{metric.capitalize()}: {value:.2f}")

# Print results
print("Results:")
print("Ratio Precision Accuracy Recall F1")
for ratio, metrics in results.items():
    print(f"{ratio:}\t{metrics['precision']:.4f}\t{metrics['accuracy']:.4f}\t{metrics['recall']:.4f}\t{metrics['f1']:.4f}")
print("Average F1",round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Accuracy",round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Precision",round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Recall",round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4))

final_results_table.loc[len(final_results_table)] = ['RL/Q-Learning', 
                                                     round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4)
                                                    ]

### Our final comparison table tabulating our results

In [ ]:
final_results_table